In [2]:
%load_ext autoreload
%autoreload 2
from multiprocessing import get_context
import os
import time
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_gadi_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
# Parent directory for saving outputs
base_local = Path("/scratch/w40/esh563/THOR_output")
year=2010
event_directories = data.gridrad.get_event_directories(year)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/dev/gridrad_{event_start.replace('-', '')}"
# if output_parent.exists():
    # shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
gridrad_parent = str(base_local / "input_data/raw")
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start,
    end=end,
    converted_options=converted_options,
    event_start=event_start,
    parent_local=gridrad_parent,
)
era5_parent = "/g/data/rt52"
era5_pl_options = data.era5.data_options(
    start=start, end=end, parent_local=era5_parent
)
kwargs = {"start": start, "end": end, "data_format": "single-levels"}
kwargs.update({"parent_local": era5_parent})
era5_sl_options = data.era5.data_options(**kwargs)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)
dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = None

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/w40/esh563/THOR_output/input_data/raw/d841006/volumes/2010'

In [4]:
times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-18 17:20:04,043 - thor.track - INFO - Beginning thor run. Saving output to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo.
2024-10-18 17:20:04,046 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-18 17:20:04,763 - thor.track - INFO - Processing 2010-01-21T12:00:00.
2024-10-18 17:20:04,765 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T12:00:00.
2024-10-18 17:20:04,768 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T120000Z.nc
2024-10-18 17:20:04,771 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-21T12:00:00.000000000.
2024-10-18 17:20:05,877 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-18 17:20:09,582 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-18 17:20:09,872 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-18 17:20:11,518 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad da

In [ ]:
num_processes = int(os.cpu_count() * 0.5)
num_processes = 6
with log.logging_listener(), get_context("spawn").Pool(
    initializer=parallel.initialize_process, processes=num_processes
) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.model_copy(), grid_options.copy()]
        args += [track_options.copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)

2024-10-18 16:18:22,376 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_0/options/data.yml
2024-10-18 16:18:22,378 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_1/options/data.yml
2024-10-18 16:18:22,386 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_0/options/grid.yml
2024-10-18 16:18:22,386 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_2/options/data.yml
2024-10-18 16:18:22,387 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_1/options/grid.yml
2024-10-18 16:18:22,390 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/interval_3/options/data.yml
2024-10-18 16:18:22,391 - thor.utils - DEBUG - Saving options to /scratch/w40/esh563/THOR_output/runs/gridrad_ga

In [3]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-18 18:19:09,756 - thor.parallel - INFO - Stitching all attribute, mask and record files.


KeyError: 'track'

In [7]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="gadi", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
kwargs = {"parallel_figure": False, "dt": 7200, "by_date": False}
visualize.attribute.mcs_series(*args, **kwargs)

2024-10-18 18:59:01,792 - thor.write.attribute - DEBUG - Writing attribute dataframe to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/velocities.csv
2024-10-18 18:59:01,796 - thor.write.attribute - DEBUG - Saving attribute metadata to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/velocities.yml
2024-10-18 18:59:01,980 - thor.write.attribute - DEBUG - Writing attribute dataframe to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/quality.csv
2024-10-18 18:59:01,984 - thor.write.attribute - DEBUG - Saving attribute metadata to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/quality.yml
2024-10-18 18:59:02,046 - thor.write.attribute - DEBUG - Writing attribute dataframe to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/classification.csv
2024-10-18 18:59:02,049 - thor.write.attribute - DEBUG - Saving attribute metadata to /scratch/w40/esh563/THOR_output/runs/gridrad_gadi_demo/analysis/classification.yml
20

KeyboardInterrupt: 